# Problem statement

Let $X \in \{0,1\}$ be a random variable, that satisfies $$prob(X=1) = p = \frac {exp(a^{T}x+b)}{1+exp(a^{T}x+b)} $$  where $x \in \mathbb{R}^n$ is a vector of variables that affect the probability, and $a$ and $b$ are known parameters. We can think of $X = 1$ as the event that a consumer buys a product, and $x$ as a vector of variables that affect the probability, e.g., advertising effort, retail price, discounted price, packaging expense, and other factors. 
The variable $x$, which we are to optimize over, is subject to a set of linear constraints, $Fx \preceq g$.

Formulate the following problems as convex optimization problems.


1)  _Maximizing buying probability_ The goal is to choose $x$ to maximize $p$.

2)  _Maximizing expected profit._ Let $c^T x+d$ be the profit derived from selling the product, which we assume is positive for all feasible x. The goal is to maximize the expected profit, which is $p\cdot(c^T x + d)$.  


In [1]:
import numpy as np

import matplotlib.pyplot as plt
import scipy.stats as sps
%matplotlib inline

In [76]:
def f_a(x, a=a, b=b):
    return -a.T * x - b
    
def f_b(x, a= a, b= b,c=c, d=d):
    return -(c.T*x + d) * np.exp(a.T*x + b) / (1. + np.exp(a.T*x + b))

Because the optimization methods work for minimizing function f, in our task we minimize -f, which is the same as maximizing f. 

a) Gradient here is constant, $p' = -a$

b) Gradient: $$p' = -a + \frac {e^{a^Tx + b}}{1 + e^{a^Tx + b}} - \frac {c}{c^Tx + d}$$

In [117]:
def grad_a(x, a=a, **kwargs):
    return -a
def grad_b(x, a=a, b=b, c=c, d=d, **kwargs):
    nom = np.exp(a.T*x + b)[0, 0]
    denom = 1 + nom
    
    if c.T*x + d <= 0:
        raise Exception('grad_b error: Divided by zero')
    return -a + nom*a/denom - c/(c.T*x + d)

a) Hesse matrix is zero, as derivative is constant.

b) Hesse matrix: $$ \frac{cc^T}{(c^Tx + d)^2}  + \frac {e^{a^Tx + b}}{(1+e^{a^Tx + b})^2}aa^T$$

In [79]:
def hesse_a(x, **kwargs):
    return 0
def hesse_b(x, a=a, b=b, c=c, d=d, **kwargs):
    nom = np.exp(a.T*x + b)[0,0]
    
    
    if c.T*x + d <= 0:
        raise Exception('grad_b error: Divided by zero')

    return c*c.T/(c.T*x+d)**2 + nom/(nom+1)**2*a*a.T

In [80]:
def grad_descent(grad, start_x, F= None, g= None, stop_precision=0.0001, epsilon=0.001, **kwargs):
    work_x = start_x
    prev_x = 0
    prev_precision = 1
    iterat = 0
    for i in range(10000):
        eps = epsilon
        iterat += 1
        prev_x = work_x
        
        if (F is not None) and (g is not None):
            it = 1
            while np.count_nonzero(g - F*(work_x - eps*grad(prev_x,**kwargs)) <= 0) > 0: #projectioin
                it += 1
                eps = 0.6 * eps
                if(it > 100000):
                    return work_x
        
        work_x = work_x - eps * grad(prev_x,**kwargs)
        prev_precision = np.max(np.absolute(prev_x - work_x))

        if prev_precision < stop_precision:
            break
    return work_x

In [118]:
def barrier_method(start_x,
                   start_t,
                   F, g,
                   grad_func,
                   hesse_func=None, 
                   optimization_method=grad_descent,
                   mu= 1.05, 
                   eps = (10.0)**(-3),
                   iterations_limit = 1000,
                   **kwargs):
    """
    Barrier method for minimizing f subject to Fx < g
    Instead of minimizing f we will minimize 
    h_t = t * f + penalty_function,
    where penalty_function is 
    - sum (from 1 to m) ln(g_i - f_i * x)
    f_i - i-th row of matrix F, g_i - i-th value in g,
    t - positive number
    start_t is passed to input, later the number is changed 
    
    Parameters:
    start_x - np.matrix with shape (n, 1)
    start_t - float
    F - np.matrix with shape (m, n)
    g - np.matrix with shape (m, 1)
    grad_func - function, computing gradient for f, 
                returns np.matrix with shape (n, 1)
                
    hesse_func - function, computing hessian for f,
                returns np.matrix with shape (n, n)
    optimization_method - chosen method of finding minimum of 
                function (f + penalty_function)
    mu - float, measure of parameter t increase
    eps - precision of computing argmin f
    """
    
    def grad_log_func(x):
        d = 1/(g - F * x)
        return F.T * d
    
    def hesse_log_func(x):
        d = 1/(g - F* x)
        D = np.diag((np.array(d.T)).reshape(np.size(d)))
        return F.T * D * D * F
    
    
    m = np.size(g, axis= 1) # dimension of g
    t = start_t
    xx = start_x
    iterations = 0
    while iterations < iterations_limit:

        def grad_x(x, **kwargs):
            """
            compute grad of h_t(x)
            """
            return t * grad_func(x, **kwargs) + grad_log_func(x)
        
        if hesse_func is None: 
            x_star = optimization_method(grad_x, xx, F= F, g=g, stop_precision=eps, **kwargs)
        
        if hesse_func is not None: # basically the case of Newton's method
            def hesse_x(x, **kwargs):
                """
                compute hessian of h_t(x)
                """
                return t * hesse_func(x, **kwargs) + hesse_log_func(x)
            
            x_star = optimization_method(grad_x, hesse_x, xx, F= F, g=g, stop_precision=eps, **kwargs)
        
        xx = x_star
        if m / t < eps:
            break
        else:
            t = mu * t
        iterations += 1
    return xx

### Quasi-Newton

In [88]:
def quasi_newton_method(grad, start_x, F= None, g=None, stop_precision=0.0001,\
                        step_size=1, iterations=10000, **kwargs):
    """
    Minimizes function using quasi Newton method
    grad - gradient of a function to minimize
    start_x - starting point
    """
    def update_inverse_hesse(prev, s, y):
        v = s - prev*y
        if v.T*y == 0 or v.T*y != v.T*y:
             raise Exception('hesse error: Divided by zero %f' % v.T*y)
        return prev + v*v.T/(v.T*y)
    
    curr_x = start_x

    inverse_hesse = np.identity(start_x.size) #hesse aproximation

    for i in range(iterations):
        eps = step_size
        
        delta_x = -inverse_hesse * grad(curr_x, **kwargs)
        
        if (F is not None) and (g is not None):
            while np.count_nonzero(g - F*(curr_x + eps * delta_x) <= 0) > 0: #projectioin
                eps = 0.6 * eps
        
        s = eps * delta_x
        y = grad(curr_x+s, **kwargs) - grad(curr_x, **kwargs)
        
        precision = np.max(np.absolute(s))
        curr_x = curr_x + s
        
        if precision < stop_precision or np.all(grad(curr_x, **kwargs) == 0):
            break
        
        inverse_hesse = update_inverse_hesse(inverse_hesse, s, y)
        
    return curr_x

#### The following example shows the advantages of using barrier method instead of simple gradient descent

In [90]:
print('Min at ', grad_descent(grad_b, start_b, F= FF, g= gg, stop_precision=0.0001, epsilon=0.001))

Min at  [[2.01093455]
 [0.6593663 ]]


In [91]:
print('For gradient descent, max = ', -f_b(np.matrix([[2.01093455],
        [0.6593663 ]])))
print('For barrier method, max = ', -f_b(np.matrix([[ 7.55725553],
     [-3.03861009]])))


For gradient descent, max =  [[6.59437475]]
For barrier method, max =  [[12.98826058]]


We see, that using barrier method gives better results.


### Newton

In [87]:
def newton_method(grad, hesse, start_x, F= None, g=None, \
                  stop_precision=0.0001, step_size=1, iterations=1000, **kwargs):
    """
    Minimizes function using Newton method
    grad - gradient of a function to minimize
    start_x - starting point
    """
    curr_x = start_x
    for i in range(iterations):
        eps = step_size
        inverse_hesse = (hesse(curr_x, **kwargs))**(-1)
        gradient = grad(curr_x, **kwargs)
        
        delta_x = - inverse_hesse * gradient
        
        if (F is not None) and (g is not None):
            while np.count_nonzero(g - F*(curr_x + eps * delta_x) <= 0) > 0: #projectioin
                eps = 0.6 * eps
        
        lambda_square = gradient.T * inverse_hesse * gradient
        if lambda_square/2 < stop_precision:
            return curr_x + eps * delta_x
        curr_x = curr_x + eps*delta_x
    return curr_x

In [93]:
print('min at', barrier_method(start_b, 0.4, FF, gg, grad_b, hesse_func= hesse_b,
               optimization_method= newton_method, mu= 10, iterations_limit = 100000))

min at [[ 7.99593409]
 [-3.33164255]]


Now we will check, if our methods work correctly for function without restrictions. As an example, we will use function $x^2 - 5x$.

In [72]:
def grad_square(x):
    return 2*x + 5
def hesse_square(x):
    return 2

print('grad-descent min =', grad_descent\
      (grad_square, np.array(0)))
print('quasi-newton min =', quasi_newton_method\
      (grad_square, np.array(0), stop_precision=0.0001, step_size=1, iterations=10000))
print('newton min =', newton_method\
      (grad_square, hesse_square,  np.array(0), iterations=10000))

grad-descent min = -2.4501943322886444
quasi-newton min = [[-2.5]]
newton min = -2.5


As we can see, all method work correctly.

First let us examine 1-D example.
$$ a = 1, b = -2, c = 2, d=2, F = 2, g = 15$$ Then the function is:
a) $$ \frac {exp(x-2)}{1+exp(x-2)}$$
b) $$ \frac {exp(x-2)}{1+exp(x-2)}(2x+2)$$

subject to $2x<15$

Because function is increasing, maximum will br at 7,5.

In [108]:
a = np.matrix([1]).T
b = -2
c = np.matrix([2]).T
d = 2
start_a = np.matrix([0]).T
start_b = np.matrix([1]).T
g = np.matrix([15]).T
F = np.matrix([[2]])
FF = np.concatenate((F, -c.T))
gg = np.vstack((g, d))

stop_precision = 0.001
epsilon = 0.01
print('Max grad descent at ', barrier_method(start_b, 0.4, \
        FF, gg, grad_b, mu= 10, iterations_limit = 100000, a=a, b=b, c=c, d=d))
print('Max quasi newton at ', barrier_method(start_b, 0.4, FF, gg, grad_b, 
               optimization_method= quasi_newton_method, mu= 10, iterations_limit = 100000, a=a, b=b, c=c, d=d))
print('Max newton at', barrier_method(start_b, 0.4, FF, gg, grad_b, hesse_func= hesse_b,
               optimization_method=newton_method, mu= 10, iterations_limit = 100000, a=a, b=b, c=c, d=d))

Max grad descent at  [[7.49825843]]
Max quasi newton at  [[7.49767651]]
Max newton at [[7.49794752]]


Now let us check if they work correctly for our function. As an example we take $$ a = (1,2), b = 1, c = (2,1), d=2, F = ((2,3),(1,0)), g = (6,8)$$ Then the function is:
a) $$ \frac {exp(x_1 + 2x_2+1)}{1+exp(x_1 + 2x_2+1)}$$
b) $$ \frac {exp(x_1 + 2x_2+1)}{1+exp(x_1 + 2x_2+1)}(2x_1 + x_2 + 2)$$

subject to $2x_1 + 3x_2 < 6$ and $x_1 < 8$

In [125]:
a = np.matrix([1,2]).T
b = 1
c = np.matrix([2,1]).T
d = 2
start_a = np.matrix([1, 1]).T
start_b = np.matrix([1, 0]).T
g = np.matrix([6, 8]).T
F = np.matrix([[2, 3],[1, 0]])

FF = np.concatenate((F, -c.T))
gg = np.vstack((g, d))

stop_precision = 0.001
epsilon = 0.01

First, for a):

In [130]:
print('Max grad descent a at ', barrier_method(start_a, 0.4, \
        F, g, grad_a, mu= 10, iterations_limit = 100000, a=a))
print('Max = ', -f_a([[-3872.71308567],
 [ 2583.80842628]],a=a))

Max grad descent a at  [[-3872.71308567]
 [ 2583.80842628]]
Max =  [[1295.90376689]]


For b):

In [122]:
%%time
print('Max grad descent at ', barrier_method(start_b, 0.4, \
        FF, gg, grad_b, mu= 10, iterations_limit = 100000, a=a, b=b, c=c, d=d))

Max grad descent at  [[ 7.55725553]
 [-3.03861009]]
CPU times: user 2.12 s, sys: 4 ms, total: 2.13 s
Wall time: 2.12 s


In [101]:
%%time
print('Max quasi newton at ', barrier_method(start_b, 0.4, FF, gg, grad_b, 
               optimization_method= quasi_newton_method, mu= 10, iterations_limit = 100000, a=a, b=b, c=c, d=d))

Max quasi newton at  [[ 7.99688143]
 [-3.33365584]]
CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 22 ms


In [102]:
%%time
print('Max newton at', barrier_method(start_b, 0.4, FF, gg, grad_b, hesse_func= hesse_b,
               optimization_method=newton_method, mu= 10, iterations_limit = 100000, a=a, b=b, c=c, d=d))

Max newton at [[ 7.99593409]
 [-3.33164255]]
CPU times: user 8 ms, sys: 12 ms, total: 20 ms
Wall time: 9.83 ms
